

---

# Part 7: Advanced Topics and Modern Networking

## Chapter 23: Advanced Wireless and Mobility

In Chapter 4, we introduced the physical layer fundamentals of wireless networking: frequencies, channels, and signal strength. We explored how Wi-Fi (IEEE 802.11) enables devices to connect without cables. However, in modern enterprises, wireless networking is far more complex than a single access point (AP) broadcasting a signal. It involves dozens or hundreds of APs, centralized management, seamless roaming, and careful design to ensure reliable coverage and performance.

This chapter delves into the advanced concepts of wireless networking. You will learn about the shift from standalone APs to controller-based architectures, where a Wireless LAN Controller (WLC) manages multiple lightweight APs. You will explore the intricacies of roaming, both at Layer 2 and Layer 3, and how devices maintain connectivity as they move. You will understand the importance of RF site surveys in designing robust wireless networks. Finally, you will be introduced to the latest Wi-Fi standards—Wi-Fi 6, 6E, and the emerging Wi-Fi 7—and the role of cellular networking (4G/5G) as a WAN connectivity option.

By the end of this chapter, you will have a comprehensive understanding of what it takes to design, deploy, and manage a modern, high-performance wireless network.

### 23.1 Wireless LAN Controllers (WLCs) and Lightweight APs (CAPWAP)

In small deployments, such as a home or a small office, **autonomous APs** are sufficient. Each autonomous AP is configured individually and operates independently. It handles all 802.11 functions: beaconing, client authentication, encryption, and forwarding traffic to the wired network.

However, this model does not scale. Imagine a large campus with 500 APs. Configuring each one individually would be a nightmare. Managing security policies, pushing firmware updates, and monitoring RF interference across 500 autonomous APs is impractical.

This led to the development of the **split-MAC architecture** , which separates the 802.11 functions between two devices:

- **Lightweight Access Points (LAPs):** These APs are "lightweight" because they handle only real-time, time-sensitive functions. They are responsible for:
    - Beaconing and probing.
    - Transmitting and receiving 802.11 frames.
    - Encrypting/decrypting traffic (if the encryption is done at the AP).
    - Sending management frames and client data to the controller.

- **Wireless LAN Controller (WLC):** The WLC is a centralized appliance (physical or virtual) that handles the non-real-time, management functions. It is responsible for:
    - **AP Management:** Configuring APs, pushing firmware updates, and monitoring AP status.
    - **Client Authentication:** Acting as the authenticator for 802.1X and other authentication methods.
    - **Security Management:** Managing encryption keys and security policies.
    - **Roaming Coordination:** Helping clients roam seamlessly between APs.
    - **RF Management:** Monitoring the RF environment, detecting interference, and automatically adjusting AP power levels and channels (a feature called Radio Resource Management or RRM).

**CAPWAP (Control and Provisioning of Wireless Access Points)**

The communication between a Lightweight AP and a WLC is governed by a protocol called **CAPWAP (Control and Provisioning of Wireless Access Points)** , defined in RFC 5415. CAPWAP is based on an older Cisco protocol called LWAPP (Lightweight Access Point Protocol) and is now an open standard.

CAPWAP creates two tunnels between the AP and the controller:
- **Control Tunnel:** This tunnel carries management traffic. It is used for configuration, firmware updates, and keep-alive messages between the AP and the WLC. The control tunnel is always encrypted for security.
- **Data Tunnel:** This tunnel carries user data traffic. When a client sends data to the internet, the AP forwards it through the CAPWAP data tunnel to the WLC. The WLC then strips off the CAPWAP header and forwards the traffic onto the wired network. The data tunnel can be optionally encrypted.

In a typical deployment, APs discover and join a WLC using various methods (DNS, DHCP options, or broadcast). Once joined, they are fully managed by the controller. If the CAPWAP tunnel to the WLC is lost, the AP will continue to operate (if configured to do so) but will lose its ability to be managed, and new clients may not be able to associate.

### 23.2 Roaming (Layer 2 and Layer 3)

One of the primary goals of a wireless network is to allow users to move freely while maintaining their network connections. This process is called **roaming**. When a client moves out of range of its current AP and associates with a new AP, it must roam. How this handoff occurs depends on whether the new AP is on the same subnet (Layer 2 roaming) or a different subnet (Layer 3 roaming).

**Layer 2 Roaming**

Layer 2 roaming occurs when a client moves between APs that are all on the same VLAN and IP subnet. This is the simplest and most common form of roaming.

- **How it works:**
    1.  The client is associated with AP1. It has an IP address on VLAN 10.
    2.  As the client moves away from AP1, the signal weakens. At some point, the client decides to reassociate with AP2, which also has a stronger signal.
    3.  The client sends a reassociation request to AP2. AP2, being a lightweight AP, forwards this request to the WLC.
    4.  The WLC has context for the client. It knows the client's MAC address, IP address, and security associations. It updates its internal database to indicate that the client is now associated with AP2.
    5.  The WLC may also inform the network switch (via a technique called "anchor forwarding" or by updating the switch's MAC table) that the client's traffic should now be sent to AP2's port.
    6.  The client retains the same IP address, and its ongoing sessions (e.g., a VoIP call, a file download) continue uninterrupted. The roaming is seamless.

**Layer 3 Roaming**

Layer 3 roaming (also called inter-subnet roaming) occurs when a client moves between APs that are on different IP subnets. This is more complex because the client's IP address is tied to its original subnet. If it simply moved to a new subnet with a new IP, all its active connections would break.

To solve this, WLCs use a technique called **tunneling** or **anchor controllers**.

- **How it works:**
    1.  A client associates with AP1 in Building A. Its IP address is 10.10.10.5/24 (VLAN 10). The WLC handling AP1 is the client's **anchor controller**.
    2.  The client moves to Building B, where APs are on a different subnet (e.g., 10.10.20.0/24). The APs in Building B may be managed by the same WLC or a different one.
    3.  When the client associates with AP2 in Building B, the WLC (or the new WLC) recognizes that this client has an IP address from a different subnet.
    4.  A tunnel (often using CAPWAP or a proprietary protocol) is established between the WLC handling AP2 and the client's anchor controller (the original WLC).
    5.  All traffic from the client is now encapsulated and sent through this tunnel back to the anchor controller. The anchor controller then forwards the traffic onto the wired network. To the rest of the network, it appears as if the client is still in Building A.
    6.  The client retains its original IP address and active sessions. The roaming is transparent.

Layer 3 roaming allows users to move across large campuses or even between buildings while maintaining connectivity, but it can introduce additional latency as traffic must be tunneled back to the anchor point.

### 23.3 RF Site Surveys and Design

A successful wireless network is not just about buying good APs and placing them randomly. It requires careful planning and design, which begins with an **RF (Radio Frequency) site survey**.

An RF site survey is the process of planning and assessing a physical environment to determine the optimal placement, configuration, and number of access points to provide the desired coverage, capacity, and performance.

**Why Conduct a Site Survey?**

- **Coverage:** Ensure there are no "dead spots" where signal is too weak for reliable connectivity.
- **Capacity:** Ensure that the network can handle the expected number of users and devices in each area. A conference room that will host 100 people needs more APs (higher capacity) than a hallway with light traffic (coverage only).
- **Interference:** Identify sources of RF interference, such as microwave ovens, cordless phones, neighboring Wi-Fi networks, or Bluetooth devices, and design the network to mitigate them.
- **Channel Planning:** Determine the optimal channel assignments for APs to minimize co-channel interference (where APs on the same channel overlap) and adjacent channel interference.
- **AP Placement:** Determine the exact locations where APs should be mounted to achieve the coverage and capacity goals.

**Types of Site Surveys:**

- **Passive Survey:** An engineer walks through the site with a laptop and a Wi-Fi adapter running survey software (like Ekahau or NetSpot). The software listens to existing Wi-Fi signals (from existing APs or other sources) and measures signal strength, noise, and interference. No test APs are deployed. This is useful for assessing an existing environment or planning a new one.
- **Active Survey:** After APs have been deployed (or temporary test APs are set up), the engineer walks the site while generating actual traffic and measuring performance metrics like throughput, latency, and packet loss. This validates that the network performs as expected under load.
- **Predictive Survey:** Using specialized software and blueprints of the building, an engineer can simulate AP placement and predict coverage and performance. This is done before any physical deployment and helps refine the design. The predictions are then validated with an on-site active survey.

**Key Factors Considered in a Site Survey:**

- **Building Materials:** Drywall, glass, wood, concrete, and metal all affect Wi-Fi signals differently. Concrete and metal cause significant attenuation.
- **AP Antennas:** The type of antenna (omnidirectional vs. directional) and its placement affect coverage patterns.
- **User Density:** High-density areas (auditoriums, lecture halls) require more APs with lower power to create smaller cells and support more users.
- **Roaming Boundaries:** APs should be placed so that there is sufficient overlap between cells to allow clients to roam seamlessly, but not so much overlap that co-channel interference becomes an issue.

### 23.4 Wi-Fi 6 (802.11ax) and Wi-Fi 7: Key Features

Wi-Fi standards have evolved continuously to provide higher speeds, better capacity, and improved efficiency. Understanding the latest standards is essential for any network professional.

**Wi-Fi 6 (802.11ax)**

Wi-Fi 6, based on the IEEE 802.11ax standard, is a significant leap forward from Wi-Fi 5 (802.11ac). It is designed not just for faster speeds, but for better performance in dense environments with many connected devices.

**Key Features of Wi-Fi 6:**

- **OFDMA (Orthogonal Frequency Division Multiple Access):** This is a game-changer. In previous Wi-Fi generations (Wi-Fi 5 and earlier), each transmission used an entire channel for a single device, even for small data packets. OFDMA divides a channel into smaller sub-channels, called Resource Units (RUs), allowing multiple devices to transmit simultaneously in the same channel. This dramatically reduces latency and improves efficiency, especially for IoT devices and smartphones that send small bursts of data.
- **MU-MIMO (Multi-User Multiple Input Multiple Output):** Wi-Fi 5 introduced MU-MIMO for downlink (AP to client). Wi-Fi 6 extends MU-MIMO to both downlink and uplink, and supports up to 8 simultaneous streams. This allows the AP to communicate with multiple devices at the same time, increasing overall network capacity.
- **1024-QAM (Quadrature Amplitude Modulation):** Wi-Fi 6 uses a higher-order modulation scheme (1024-QAM) compared to Wi-Fi 5's 256-QAM. This packs more data into the same signal, resulting in a theoretical speed increase of about 25% (up to 9.6 Gbps aggregate).
- **Target Wake Time (TWT):** This feature allows devices to negotiate when they will wake up to send or receive data. This significantly improves battery life for IoT devices and smartphones by allowing them to sleep for longer periods.
- **BSS Coloring:** In dense environments, APs on the same channel can interfere with each other. BSS Coloring adds a "color" (a number) to the preamble of a transmission. A device can ignore transmissions from APs with a different color, reducing unnecessary waiting and improving spatial reuse.

**Wi-Fi 6E (Extended)**

Wi-Fi 6E is an extension of Wi-Fi 6 that operates in the newly opened **6 GHz band**. This provides a massive amount of additional spectrum (up to 1200 MHz in some regions) that is currently free from interference from legacy devices. Wi-Fi 6E offers wider channels (up to 160 MHz) and less congestion, enabling even higher performance and lower latency.

**Wi-Fi 7 (802.11be) - The Emerging Standard**

Wi-Fi 7 is the next generation, still under development as this book is written, but expected to be finalized soon. It aims to build on Wi-Fi 6E and deliver "Ethernet-like" performance for wireless.

**Anticipated Key Features of Wi-Fi 7:**

- **320 MHz Channels:** Doubling the maximum channel width from 160 MHz to 320 MHz, enabling even higher peak data rates.
- **4096-QAM:** A further increase in modulation complexity, packing more data into each signal for a theoretical 20% speed boost over Wi-Fi 6's 1024-QAM.
- **Multi-Link Operation (MLO):** This is a major new feature. MLO allows a device to simultaneously send and receive data across multiple bands (e.g., 2.4 GHz, 5 GHz, and 6 GHz) and channels. This increases throughput, reduces latency, and provides greater reliability. If one link is congested or experiences interference, traffic can be sent over another link.
- **Enhanced MU-MIMO:** Supporting up to 16 spatial streams (up from 8 in Wi-Fi 6), further increasing capacity.

Wi-Fi 7 is expected to deliver speeds exceeding 40 Gbps and will be crucial for emerging applications like high-definition video streaming, augmented reality (AR), virtual reality (VR), and industrial IoT.

### 23.5 Cellular Networking (4G/LTE, 5G) as WAN Connectivity

While Wi-Fi dominates local area connectivity, cellular networks (4G/LTE and 5G) are playing an increasingly important role in wide area networking, especially as a WAN connectivity option for businesses.

**4G/LTE (Long-Term Evolution)**

4G/LTE is the current mature standard for cellular data. It provides high-speed mobile broadband and is widely available. For businesses, LTE can be used as:

- **Primary WAN for Remote/Branch Offices:** In locations where wired broadband (DSL, cable, fiber) is unavailable or unreliable, a 4G/LTE router with a SIM card can provide primary internet connectivity.
- **Failover/Backup WAN:** In mission-critical locations, a 4G/LTE connection can serve as a backup if the primary wired WAN link fails. Many enterprise routers have built-in LTE failover capabilities.

**5G (Fifth Generation)**

5G is the next generation of cellular technology, designed to go far beyond faster smartphones. It is a foundational technology for the future of connectivity, with three main use cases:

- **Enhanced Mobile Broadband (eMBB):** This is what most people think of—much faster speeds (gigabit-class) for applications like high-definition video streaming, VR/AR, and large file downloads.
- **Ultra-Reliable Low-Latency Communications (URLLC):** 5G is designed to provide extremely low latency (as low as 1 millisecond) and very high reliability. This enables mission-critical applications like autonomous vehicles, remote surgery, and industrial automation.
- **Massive Machine-Type Communications (mMTC):** 5G is designed to support a massive number of low-power, low-data-rate IoT devices (sensors, meters, trackers) per square kilometer, far more than 4G can handle.

**5G for Enterprise WAN:**

5G is becoming a compelling alternative to traditional wired WAN connections like MPLS and broadband. With the right service level agreements (SLAs), 5G can provide:
- **High Bandwidth:** Comparable to fiber broadband.
- **Low Latency:** Suitable for real-time applications.
- **Network Slicing:** A key 5G feature that allows service providers to create virtual, end-to-end network slices with guaranteed performance characteristics. An enterprise could lease a "slice" of the 5G network dedicated to its traffic, providing a level of predictability previously only available with private wired lines.

As 5G networks mature, they will increasingly be integrated into enterprise SD-WAN solutions, offering a flexible and high-performance WAN option alongside MPLS and broadband.

---

### Chapter 23: Hands-On Challenge

Let's explore some of these advanced wireless concepts.

1.  **Analyze Your Home Wi-Fi (with a Smartphone App):**
    - Download a Wi-Fi analyzer app on your smartphone (e.g., "Wi-Fi Analyzer" on Android, or "NetSpot" on iOS/Android).
    - Walk around your home or office and observe the signal strength (RSSI) of your network. Note how it changes as you move away from the AP or go through walls. This is a basic, informal site survey.
    - Observe the channels used by your network and your neighbors' networks. In the 2.4 GHz band, are there many APs on the same channel? This is co-channel interference.

2.  **Explore Controller-Based Wi-Fi (if you have access):**
    - If you are on a corporate or university network, you are likely connecting to a controller-based Wi-Fi system. You won't have direct access to the WLC, but you can observe its effects.
    - Walk from one building to another while on a continuous ping to a stable address (like `8.8.8.8`). Do you see any dropped packets? The number of drops can give you a sense of the roaming performance.

3.  **Research Wi-Fi 6 and 5G in Your Area:**
    - Check if your smartphone or laptop supports Wi-Fi 6. (Most modern devices do). In the Wi-Fi settings, you might see "Wi-Fi 6" or "802.11ax" mentioned.
    - Check your cellular carrier's coverage map for 5G in your area. Are you able to get a 5G signal on your phone? Run a speed test on 4G vs. 5G to see the difference (if available).

4.  **Simulate a Basic Wireless Network in Packet Tracer:**
    - Packet Tracer includes wireless devices, including a Wireless LAN Controller and Lightweight APs.
    - Create a topology with a WLC, a couple of Lightweight APs, and a few wireless clients. Configure the WLC and have the APs join it.
    - Observe how the clients associate with the APs. This is a great way to understand the controller-AP relationship.

5.  **Read About Cellular WAN Solutions:**
    - Search for enterprise routers with 4G/5G capabilities. Look at products from Cisco (e.g., Cisco IR1100, Cradlepoint) or other vendors.
    - Read case studies about businesses using 5G as a primary WAN connection. Understanding these real-world use cases will help you appreciate the technology's potential.

---

This chapter has elevated your understanding of wireless networking from basic principles to advanced, enterprise-grade concepts. You now understand the split-MAC architecture and the role of WLCs and CAPWAP, the intricacies of Layer 2 and Layer 3 roaming, the importance of RF site surveys, and the key features of Wi-Fi 6, 6E, and the emerging Wi-Fi 7. You have also been introduced to the role of 4G/5G cellular as a vital WAN connectivity option.

In the next and final chapter of Part 7, we will bring together everything you have learned in this book and apply it to the real world. **Chapter 24: Network Troubleshooting Methodology** will provide you with a structured, layered approach to diagnosing and resolving network problems, along with a comprehensive toolkit of essential commands and techniques.